In [1]:
import os.path
import sys
import numpy as np
import pandas as pd

# Provide paths to the respective folders
script_folder = r'D:\UoK\OneDrive - University of Kentucky\github\Transit_ridership\transit_ridership_decline\Factors and Ridership Data\code'
load_data = r'D:\UoK\OneDrive - University of Kentucky\github\Transit_ridership\transit_ridership_decline\Factors and Ridership Data\Model Estimation\Est4'
output_folder = r'D:\UoK\OneDrive - University of Kentucky\github\Transit_ridership\transit_ridership_decline\Factors and Ridership Data\Script Outputs'
folder_path = ''
file_name = ''
os.chdir(r'D:\UoK\OneDrive - University of Kentucky\github\transit_ridership_decline\Factors and Ridership Data\Shared_dataset')
df = pd.read_csv('merged_df_3.csv')
modes = df['Mode'].unique()